In [1]:
import requests, os, datetime, json
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

Note
- File hierarchy: data -> tables -> file

In [2]:
# Change these -------------
start_year = 2001
end_year = 2021 # one after last year you want
league = 'ENG.1'
folder_name = 'epl' # put league or competition here
# --------------------------

file_name = '{}_{}_tables.csv'.format(start_year, end_year-1)
url = "https://www.espn.com/soccer/standings/_/league/{}/season/{}"

# GP W D L GF GA GD P
cols = ['Place', 'Team', 'GP', 'W', 'D', 'L', 'GF', 'GA', 'GD', 'P'] # may need to change if other leagues have other cols
table_df = pd.DataFrame([], columns=cols)

for year in range(start_year, end_year): # possibly change these
    r = requests.get(url.format(league, year))
    r.raise_for_status()
    soup = BeautifulSoup(r.text, "html.parser")

    teams_soup = soup.find_all('table')[0].find_all('div', class_='team-link flex items-center clr-gray-03')
    
    teams = []
    for team in teams_soup:
        teams.append([team.contents[0].text, team.find('img').get('title')])

    stats = soup.find_all('span', class_='stat-cell')

    # convert to np array and reshape
    stats = [item.text for item in stats]
    np_stats = np.array(stats).reshape(-1, 8)

    final_arr = np.concatenate((np.array(teams).reshape(-1, 2), np_stats), axis=1)
    temp_df = pd.DataFrame(final_arr, columns=cols)
    temp_df['Year'] = year
    table_df = table_df.append(temp_df, ignore_index=True)

In [8]:
# if folder doesn't exist, make it
tpath = os.path.join('data', folder_name, 'tables')
if not os.path.exists(tpath):
    os.makedirs(tpath)

table_df['Year'] = table_df['Year'].astype(int)
table_df.to_csv(os.path.join('data', folder_name, 'tables', file_name), index=False)

In [9]:
table_df.tail()

,Place,Team,GP,W,D,L,GF,GA,GD,P,Year
395,16,Brighton & Hove Albion,32,7,13,12,33,38,-5,34,2020
396,17,Burnley,32,8,9,15,26,45,-19,33,2020
397,18,Fulham,33,5,12,16,25,43,-18,27,2020
398,19,West Bromwich Albion,32,5,9,18,28,62,-34,24,2020
399,20,Sheffield United,32,4,2,26,17,56,-39,14,2020
